In [7]:
import torch 
import torch.nn as nn

import time 
import copy
from tqdm.notebook import trange, tqdm

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.transforms as transformss

from LeNet3 import ConvolutionalNet
from LeNet3CBAM import ConvolutionalNetCBAM as ConvolutionalNetCBAM
from CBAM import CBAM as cbam

from functions import train, evaluate, confusion
#importing comet
from comet_ml import Experiment

In [8]:
#Initialising GereralTorch class

experiment = Experiment(project_name = "LeNetvCBAM-CIFAR", workspace = "leothesouthafrican", api_key = "VZwpcMJ9rL4nXJpVWl1yZUh5X")


#Report multiple hyperparameters using a dictionary
hyper_params = {
    'learning_rate': 0.001,
    'num_epochs': 30,
    'batch_size': 32,
    'image_size': 28,
    'image_channels': 3,
    'output_size': 10,
    'num_layers': 5,
    'train_val_split': 0.90,
    'device': 'cpu',
    'model_name': 'Basic CNN Model',
    'criterion': 'CrossEntropyLoss',
    'optimizer': 'Adam',
    'dataset': 'CIFAR10',
    'best_model_path': 'CIFAR_10_Basic.pt',
}

#Setting the device
device = torch.device(hyper_params['device'])

# Loading model
model = ConvolutionalNet(input_channels = hyper_params['image_channels'], num_classes = hyper_params['output_size'], image_size=hyper_params['image_size']) #Instantiating the model and sending it to the device
model.to(device)

# Setting the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device) #Setting the loss function
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['learning_rate']) #Setting the optimizer

# Adding model parameters to comet
for name, param in model.named_parameters():
    hyper_params[name] = param

# Logging the hyperparameters to comet
experiment.log_parameters(hyper_params)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/leothesouthafrican/lenetvcbam-cifar/b9f42dfa01af47bc87f5c03802d1a031
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_acc [245]  : (0.03125, 0.4375)
COMET INFO:     train_loss [245] : (1.6120774745941162, 2.597555160522461)
COMET INFO:   Parameters:
COMET INFO:     batch_size      : 32
COMET INFO:     best_model_path : CIFAR_10_Basic.pt
/Users/leo/miniforge3/envs/torch-gpu/lib/python3.8/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS 

---Model Summary---
ConvolutionalNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv3): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv5): Sequential(
    (0): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e

In [9]:
#defining transforms

train_transform = transforms.Compose([
                transforms.Resize((hyper_params['image_size'],hyper_params['image_size'])),
                transforms.ToTensor(),
                #transforms.RandomRotation(15),
                transforms.RandomHorizontalFlip(0.3),
                #transforms.RandomVerticalFlip(),
                #transforms.RandomErasing(),
                #transforms.RandomCrop(hyper_params['image_size'], padding=4),
            ])

test_transform = transforms.Compose([
                transforms.Resize((hyper_params['image_size'],hyper_params['image_size'])),
                transforms.ToTensor()
            ])

In [10]:
#Loading the CIFAR10 dataset


train_dataset = datasets.CIFAR10(root='data', train=True, transform=train_transform, download=True)
test_dataset = datasets.CIFAR10(root='data', train=False, transform=test_transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=hyper_params['batch_size'],
                                            shuffle=True,
                                            num_workers = 4)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=hyper_params['batch_size'],
                                            shuffle=False,
                                            num_workers = 4)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
#Splitting the train data into train and validation
n_train = int(len(train_dataset) * hyper_params['train_val_split'])
n_val = len(train_dataset) - n_train

train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [n_train, n_val])

val_data = copy.deepcopy(val_dataset)
val_data.dataset.transform = test_transform #change the transform to test_transform to get the validation data in the same format as the test data

#defining the val loader
val_loader = torch.utils.data.DataLoader(val_data,
                                            batch_size=hyper_params['batch_size'],
                                            shuffle=False)
                                            

print('Train data size: ', len(train_dataset))
print('Validation data size: ', len(val_dataset))
print('Test data size: ', len(test_dataset))

Train data size:  45000
Validation data size:  5000
Test data size:  10000


In [12]:
with experiment.train():
    train(hyper_params['num_epochs'], model, criterion, optimizer, train_loader, val_loader, hyper_params['best_model_path'], device, experiment)

Begin training...


100%|██████████| 157/157 [00:11<00:00, 14.05it/s]


	Train Loss: 1.795 | Train Acc: 31.25%
Epoch: 02 | Epoch Time: 6m 32s
	 Val. Loss: 1.648 |  Val. Acc: 0.39%


100%|██████████| 157/157 [00:10<00:00, 14.66it/s]


	Train Loss: 1.763 | Train Acc: 37.50%
Epoch: 03 | Epoch Time: 6m 42s
	 Val. Loss: 1.570 |  Val. Acc: 0.42%


...

	Train Loss: 0.655 | Train Acc: 81.25%
Epoch: 49 | Epoch Time: 6m 10s
	 Val. Loss: 0.941 |  Val. Acc: 0.67%


100%|██████████| 157/157 [00:10<00:00, 15.58it/s]


	Train Loss: 1.101 | Train Acc: 50.00%
Epoch: 50 | Epoch Time: 6m 5s
	 Val. Loss: 0.881 |  Val. Acc: 0.69%


100%|██████████| 157/157 [00:10<00:00, 15.53it/s]

	Train Loss: 1.010 | Train Acc: 50.00%
Epoch: 51 | Epoch Time: 6m 5s
	 Val. Loss: 0.970 |  Val. Acc: 0.66%


In [13]:
with experiment.test():

    model.load_state_dict(torch.load(hyper_params['best_model_path']))

    test_loss, test_acc = evaluate(model, test_loader, criterion, device, experiment)

    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

100%|██████████| 313/313 [00:20<00:00, 15.42it/s]

Test Loss: 0.945 | Test Acc: 66.30%


In [14]:
confusion(model, test_loader, device = device, experiment=experiment)

In [15]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/leothesouthafrican/lenetvcbam-cifar/55a55f3a24e54d12a3cad9250ae337a0
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_acc [313]      : (0.23961661341853036, 66.30391373801918)
COMET INFO:     test_loss [313]     : (0.002068622995870182, 0.9451658742877241)
COMET INFO:     train_acc [78150]   : (0.03125, 0.90625)
COMET INFO:     train_loss [78150]  : (0.5056789517402649, 2.398590326309204)
COMET INFO:     train_val_acc [50]  : (0.3922, 0.693)
COMET INFO:     train_val_loss [50] : (0.8805941884305067, 1.6483489912786302)
COMET INFO:   Parameters:
COMET INFO:     batch_size      : 32
COMET INFO:     best_model_path : CIFAR_10_Basic.pt
COMET INFO:     conv1.0.bias    : Parameter containing:
tensor([-0.1668,  0.0957, -0.3220,  0.1411, -0